# Image Dataset Exploration with Machine Learning

## Copyright notice

This version (c) 2019 Fabian Offert, [MIT License](LICENSE).

## Imports

In [57]:
import numpy as np
from PIL import Image
import pickle

import sys
sys.path.append('lib/')
from esu_util import show_img, np_PIL, PIL_np

import os
from tqdm import tnrange, tqdm_notebook
import umap
import scipy.ndimage

import torch as t
from torchvision import transforms, datasets
import torchvision as tv
import torch.nn as nn

Read image files in our dataset.

In [58]:
img_files = []
for root, dirs, files in os.walk('faces'):
    for file in files:
        if file.endswith('.jpg'):
            img_files.append(f'{root}/{file}')
print(f'{len(img_files)} images found')

20450 images found


## Extract features

We will use the VGG19 architecture in our experiments from now on. Here, we will import a fully-trained model trained on the ImageNet dataset. The architecture requires that we resize our input images to 224x224 pixels first, which we accomplish by using `torchvision.transforms`. As our *feature vector*, we will use the *first fully connected layer* of VGG19 which has 4096 dimensions.

First, however, we need to check if we have a GPU with CUDA at our disposal.

In [59]:
device = t.device("cuda" if t.cuda.is_available() else "cpu")

In [60]:
img_size = 224
feature_size = 4096 # VGG19 fc1
extractor = tv.models.vgg19(pretrained=True).to(device) # Load the model and move to the GPU, if available
print(extractor) # Show the model architecture
extractor.classifier = nn.Sequential(*list(extractor.classifier.children())[:5]) # VGG19 fc1
extractor.eval() # Set model to evaluation mode, we are not training anything

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

This is our feature extractor function. It simply loads an image file, pushes it through the network and returns the values of fc1 as a numpy matrix (of, logically, size 4096).

In [61]:
def get_features(file):
    img = Image.open(file)
    img = img.convert('RGB') # Some images are b/w so always convert to RGB (color)
    preprocess = tv.transforms.Compose([transforms.Resize((img_size, img_size)),
                                        tv.transforms.ToTensor()])
    input = preprocess(img).unsqueeze(0).cuda() # Prepare image for network
    tensor = extractor.forward(input) # Push image through network
    tensor = tensor.data.squeeze() # Remove batch dimension from result
    x = tensor.cpu().numpy() # Move to CPU, if on GPU, and convert to NumPy matrix
    return x # Return to calling function

Now we can extract the fatures for *all* our images!

In [62]:
features = np.zeros((len(img_files), feature_size))
print('Extracting features')
for i, file in tqdm_notebook(enumerate(img_files), total=len(img_files)):
    features[i] = get_features(file)

Extracting features


Let's peek into the results:

In [63]:
print(features.shape)

(20450, 4096)


Now to the most important part: dimensionality reduction. We will use the pretty recent UMAP algorithm to reduce our array down from 4096 dimensions to just 2: this way we can plot each datapoint in Euclidian space. UMAP, and related algorithms like T-SNE and PCA, are very good at *preserving relations between datapoints*. Thus, while a 2D-plot of our feature vectors is not a faithful and complete *visualization*, it is the best possible *approximation* in human-legible terms. Thankfully, with the `umap-learn` library, dimensionality reduction is fast and done in one line, ever for very large datasets, like the MoMA dataset.

In [ ]:
reducer = umap.UMAP()
features_UMAP = reducer.fit_transform(features)

In [65]:
print(features_UMAP.shape)

(20450, 2)


## Plot images

Now we can plot our dataset. We first define a function to do that for us, then feed it all the information.

In [66]:
def plot(img_files, features_UMAP, thumb_size=100, unit=1500):
    
    # Find max. and min. feature values
    value_max = np.max(features_UMAP)
    value_min = np.min(features_UMAP)
    
    # Calculate size of the plot based on these values
    canvas_size = int((abs(value_max) + abs(value_min)) * unit) + thumb_size # Images are anchored at upper left corner
    print(f'Canvas size: {canvas_size}')

    # Define plot as empty (white) canvas
    canvas = np.ones((canvas_size, canvas_size, 3), dtype=np.uint8) * 255

    print('Plotting images...')
    for i, img_file in enumerate(tqdm_notebook(img_files)):
        
        # Read image
        img = Image.open(img_file)
        img = img.convert('RGB') # Some images are grayscale so always convert
        img.thumbnail((thumb_size, thumb_size), Image.ANTIALIAS)
        img = PIL_np(img)
        
        # Read features and calculate x,y
        y = int((features_UMAP[i,0] + abs(value_min)) * unit)
        x = int((features_UMAP[i,1] + abs(value_min)) * unit)    
        
        # Plot image
        canvas[y:y+img.shape[0],x:x+img.shape[1],:] = img

    # Save plot
    return np_PIL(canvas)

In [ ]:
show_img(plot(img_files, features_UMAP))

## <font color='red'>Exercises</font>

1. Explore your own dataset.
2. Play with the visualization parameters.
3. Experiment with the feature extractor - would it be useful to use a different/differently trained network, or a different layer?